In [1]:
from scapy.all import rdpcap
import pandas as pd

# Function to extract relevant packet details
def extract_packet_info(pcap_file):
    packets = rdpcap(pcap_file)  # Read the PCAP file
    packet_data = []

    for pkt in packets:
        packet_info = {}
        
        # Basic packet info
        packet_info['timestamp'] = float(pkt.time)  # Ensure timestamp is a float
        packet_info['length'] = len(pkt)            # Packet length in bytes
        
        # Ethernet layer (if present)
        if 'Ethernet' in pkt:
            packet_info['src_mac'] = pkt['Ethernet'].src
            packet_info['dst_mac'] = pkt['Ethernet'].dst
            packet_info['eth_type'] = hex(pkt['Ethernet'].type)

        # IP layer (if present)
        if 'IP' in pkt:
            packet_info['src_ip'] = pkt['IP'].src
            packet_info['dst_ip'] = pkt['IP'].dst
            packet_info['protocol'] = pkt['IP'].proto  # e.g., 1=ICMP, 6=TCP, 17=UDP
            packet_info['ttl'] = pkt['IP'].ttl

        # ICMP layer (if present, for ping)
        if 'ICMP' in pkt:
            packet_info['icmp_type'] = pkt['ICMP'].type  # e.g., 8=request, 0=reply
            packet_info['icmp_code'] = pkt['ICMP'].code

        # Add packet info to list
        packet_data.append(packet_info)

    return packet_data

# Main execution
if __name__ == "__main__":
    # Specify your PCAP file
    pcap_file = "h2_capture.pcap"  # Adjust path if needed
    
    try:
        # Extract packet data
        packet_data = extract_packet_info(pcap_file)
        
        # Convert to Pandas DataFrame
        df = pd.DataFrame(packet_data)
        
        # Convert timestamp to human-readable format
        if not df.empty and 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
        
        # Display the DataFrame
        print("Packet DataFrame:")
        print(df)
        
        # Basic analysis
        print("\nSummary Statistics:")
        print(df.describe())
        
        # Filter for ICMP packets (ping traffic)
        if 'protocol' in df.columns:
            icmp_packets = df[df['protocol'] == 1]  # ICMP protocol number is 1
            print("\nICMP Packets:")
            print(icmp_packets[['timestamp', 'src_ip', 'dst_ip', 'icmp_type', 'icmp_code']])
        
        # Optional: Save to CSV for further analysis
        df.to_csv("h2_capture.csv", index=False)
        print("\nSaved to h2_capture.csv")
        
    except Exception as e:
        print(f"Error processing PCAP file: {e}")

Packet DataFrame:
                       timestamp  length            src_mac  \
0  2025-03-21 19:03:31.096803072      70  00:00:00:00:01:02   
1  2025-03-21 19:03:31.098797056      70  0e:f8:e2:0b:17:e4   
2  2025-03-21 19:03:31.099324928      70  00:00:00:00:01:02   
3  2025-03-21 19:03:31.354028032      70  00:00:00:00:01:01   
4  2025-03-21 19:03:31.593831168      98  00:00:00:00:01:01   
5  2025-03-21 19:03:31.593853952      98  00:00:00:00:01:02   
6  2025-03-21 19:03:31.864822016      70  4a:03:cb:a5:15:fa   
7  2025-03-21 19:03:31.866151168      70  4a:03:cb:a5:15:fa   
8  2025-03-21 19:03:32.595527936      98  00:00:00:00:01:01   
9  2025-03-21 19:03:32.595545088      98  00:00:00:00:01:02   
10 2025-03-21 19:03:33.597240832      98  00:00:00:00:01:01   
11 2025-03-21 19:03:33.597259008      98  00:00:00:00:01:02   
12 2025-03-21 19:03:34.598939904      98  00:00:00:00:01:01   
13 2025-03-21 19:03:34.598958080      98  00:00:00:00:01:02   
14 2025-03-21 19:03:35.600656896     

In [10]:
from scapy.all import rdpcap
import pandas as pd
import numpy as np

# Function to extract relevant packet details
def extract_packet_info(pcap_file):
    packets = rdpcap(pcap_file)
    packet_data = []

    for pkt in packets:
        packet_info = {}
        packet_info['timestamp'] = float(pkt.time)
        packet_info['length'] = len(pkt)

        if 'Ethernet' in pkt:
            packet_info['src_mac'] = pkt['Ethernet'].src
            packet_info['dst_mac'] = pkt['Ethernet'].dst
            packet_info['eth_type'] = hex(pkt['Ethernet'].type)

        if 'IP' in pkt:
            packet_info['src_ip'] = pkt['IP'].src
            packet_info['dst_ip'] = pkt['IP'].dst
            packet_info['protocol'] = pkt['IP'].proto
            packet_info['ttl'] = pkt['IP'].ttl

        if 'ICMP' in pkt:
            packet_info['icmp_type'] = pkt['ICMP'].type
            packet_info['icmp_code'] = pkt['ICMP'].code

        packet_data.append(packet_info)
    return packet_data

# Function to calculate RTT stats
def calculate_rtt_stats(df):
    icmp_req = df[df['icmp_type'] == 8].sort_values('timestamp')
    icmp_rep = df[df['icmp_type'] == 0].sort_values('timestamp')

    rtts = []
    for _, req in icmp_req.iterrows():
        rep = icmp_rep[
            (icmp_rep['src_ip'] == req['dst_ip']) &
            (icmp_rep['dst_ip'] == req['src_ip']) &
            (icmp_rep['timestamp'] > req['timestamp'])
        ]
        if not rep.empty:
            rtt = rep.iloc[0]['timestamp'] - req['timestamp']
            rtts.append(rtt * 1000)  # convert to ms

    if rtts:
        rtts = np.array(rtts)
        print("\nRTT Statistics (ms):")
        print(f"Min:  {rtts.min():.6f}")
        print(f"Avg:  {rtts.mean():.6f}")
        print(f"Max:  {rtts.max():.6f}")
        print(f"Mdev: {rtts.std():.6f}")

        print("\nLatency Statistics (ms) (RTT/2):")
        latencies = rtts / 2
        print(f"Min:  {latencies.min():.6f}")
        print(f"Avg:  {latencies.mean():.6f}")
        print(f"Max:  {latencies.max():.6f}")
        print(f"Mdev: {latencies.std():.6f}")
    else:
        print("\nNo RTT samples found.")

# Function to calculate bandwidth stats
def calculate_bandwidth_stats(df):
    df = df.sort_values("timestamp")
    df['timestamp_diff'] = df['timestamp'].diff()
    df['bandwidth'] = df['length'] / df['timestamp_diff'].replace(0, np.nan)

    bandwidth_values = df['bandwidth'].dropna()
    if not bandwidth_values.empty:
        print("\nBandwidth Statistics (bytes/sec):")
        print(f"Min:  {bandwidth_values.min():.2f}")
        print(f"Avg:  {bandwidth_values.mean():.2f}")
        print(f"Max:  {bandwidth_values.max():.2f}")
        print(f"Mdev: {bandwidth_values.std():.2f}")
    else:
        print("\nNot enough data to calculate bandwidth statistics.")

# Main execution
if __name__ == "__main__":
    pcap_file = "h2_capture.pcap"  # Replace if needed

    try:
        packet_data = extract_packet_info(pcap_file)
        df = pd.DataFrame(packet_data)

        if not df.empty and 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
            df['timestamp'] = df['timestamp'].astype(np.int64) / 1e9  # back to float seconds

        print("Packet DataFrame:")
        print(df)

        print("\nSummary Statistics:")
        print(df.describe())

        # Filter ICMP packets and calculate RTT/Latency
        if 'protocol' in df.columns and 'icmp_type' in df.columns:
            icmp_df = df[df['protocol'] == 1]
            print("\nICMP Packets:")
            print(icmp_df[['timestamp', 'src_ip', 'dst_ip', 'icmp_type', 'icmp_code']])
            calculate_rtt_stats(icmp_df)

        # Calculate Bandwidth stats
        calculate_bandwidth_stats(df)

        df.to_csv("h2_capture.csv", index=False)
        print("\nSaved to h2_capture.csv")

    except Exception as e:
        print(f"Error processing PCAP file: {e}")


Packet DataFrame:
       timestamp  length            src_mac            dst_mac eth_type  \
0   1.742588e+09      70  00:00:00:00:01:02  33:33:00:00:00:02   0x86dd   
1   1.742588e+09      70  00:00:00:00:01:02  33:33:00:00:00:02   0x86dd   
2   1.742588e+09      70  1e:95:fe:ed:32:bd  33:33:00:00:00:02   0x86dd   
3   1.742588e+09      70  00:00:00:00:01:01  33:33:00:00:00:02   0x86dd   
4   1.742588e+09      70  1e:95:fe:ed:32:bd  33:33:00:00:00:02   0x86dd   
5   1.742588e+09      70  46:ee:dd:d4:f4:9d  33:33:00:00:00:02   0x86dd   
6   1.742588e+09      98  00:00:00:00:01:01  00:00:00:00:01:02    0x800   
7   1.742588e+09      98  00:00:00:00:01:02  00:00:00:00:01:01    0x800   
8   1.742588e+09      98  00:00:00:00:01:01  00:00:00:00:01:02    0x800   
9   1.742588e+09      98  00:00:00:00:01:02  00:00:00:00:01:01    0x800   
10  1.742588e+09      98  00:00:00:00:01:01  00:00:00:00:01:02    0x800   
11  1.742588e+09      98  00:00:00:00:01:02  00:00:00:00:01:01    0x800   
12  1.7